# Benchmarks

## Initialize

In [1]:
%load_ext autoreload
%autoreload 2

import os
import math
import pathlib
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from IPython.display import clear_output

import warnings
from lifelines.utils import CensoringType
from lifelines.utils import concordance_index

In [2]:
node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: 
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

project_label = "22_retina_phewas"
project_path = f"{base_path}/results/projects/{project_label}"
figure_path = f"{project_path}/figures"
output_path = f"{project_path}/data"

pathlib.Path(figure_path).mkdir(parents=True, exist_ok=True)
pathlib.Path(output_path).mkdir(parents=True, exist_ok=True)

##### BEGIN ADAPT #####
# second best model
# wandb_name = 'aug++_convnext_s_mlp'
# wandb_id = '8ngm6apd'
# best model
wandb_name = 'aug++_convnext_s_mlp+'
wandb_id = '3p3smraz'
partitions = [0] # [i for i in range(22)]
##### END   ADAPT #####

experiment = wandb_id
experiment_path = f"{output_path}/{experiment}"
pathlib.Path(experiment_path).mkdir(parents=True, exist_ok=True)

/sc-projects/sc-proj-ukb-cvd


In [3]:
import pandas as pd
endpoints = sorted([l.replace('_prevalent', '') for l in list(pd.read_csv('/sc-projects/sc-proj-ukb-cvd/results/projects/22_medical_records/data/220413/frequent_endpoints.csv').endpoint.values)])

In [4]:
endpoint_defs = pd.read_feather(f"{output_path}/phecode_defs_220306.feather").query("endpoint==@endpoints").sort_values("endpoint").set_index("endpoint")
endpoint_defs.head()

,phecode,phecode_string,phecode_category,sex,ICD10_only,phecode_top,leaf
endpoint,,,,,,,
OMOP_4306655,4306655,All-Cause Death,Death,Both,NaN,None,NaN
phecode_052,052,Herpesvirus,ID,Both,0.0,052,0.0
phecode_052-1,052.1,Herpes simplex,ID,Both,0.0,052,1.0
phecode_052-3,052.3,Varicella zoster virus,ID,Both,0.0,052,0.0
phecode_052-32,052.32,Herpes zoster,ID,Both,0.0,052,1.0


In [5]:
data_covariates = pd.read_feather("/sc-projects/sc-proj-ukb-cvd/data/2_datasets_pre/211110_anewbeginning/baseline_covariates_211209.feather")[["eid", "sex_f31_0_0"]].set_index("eid")
data_covariates.head()

,sex_f31_0_0
eid,
1000018,Female
1000020,Male
1000037,Female
1000043,Male
1000051,Female


In [ ]:
# TODO: baseline_outcomes_long_220412 or baseline_outcomes_wide_220301 or baseline_outcomes_wide_220306.feather?
data_outcomes = pd.read_feather(f"{output_path}/baseline_outcomes_long_220412.feather").set_index("eid")
data_outcomes.head()

,endpoint,prev,event,time
eid,,,,
1000018,OMOP_4306655,False,False,11.866089
1000020,OMOP_4306655,False,False,13.596446
1000037,OMOP_4306655,False,False,12.868163
1000043,OMOP_4306655,False,False,12.309629
1000051,OMOP_4306655,False,False,15.291210


In [7]:
data_all = data_outcomes.merge(data_covariates, left_index=True, right_index=True, how="left").reset_index(drop=False).set_index("endpoint")

In [8]:
data_all.head()

,eid,prev,event,time,sex_f31_0_0
endpoint,,,,,
OMOP_4306655,1000018,False,False,11.866089,Female
phecode_001,1000018,False,False,11.866089,Female
phecode_002,1000018,False,False,11.866089,Female
phecode_002-1,1000018,False,False,11.866089,Female
phecode_003,1000018,False,False,11.866089,Female


In [9]:
data_dict = {e: df.reset_index(drop=True).set_index("eid") for e, df in data_all.groupby('endpoint')}
#data_dict.keys()

In [10]:
#endpoints

In [11]:
endpoint_defs.sex.unique()

array(['Both', 'Female', 'Male'], dtype=object)

In [12]:
def get_eligable_eids(data_dict, endpoint):

    data_temp = data_dict[endpoint]
    eligibility = endpoint_defs.loc[endpoint]["sex"]
    
    if eligibility == "Both": 
        eids_incl = data_temp.copy().query(f"prev==0").index.to_list()
    else:
        eids_incl = data_temp.copy().query(f"prev==0&sex_f31_0_0==@eligibility").index.to_list()
        
    return {"endpoint": endpoint, 
            "n_eids": len(eids_incl), 
            "eid_list": eids_incl}

In [13]:
#long_endpoints_unique = data_outcomes['endpoint'].unique()
#endpoints_in_long = [e for e in endpoints if e in long_endpoints_unique]

d_list = [get_eligable_eids(data_dict, endpoint) for endpoint in tqdm(endpoints)] 
eid_df = pd.DataFrame.from_dict(d_list)

  0%|          | 0/498 [00:00<?, ?it/s]

In [14]:
#eid_df.set_index("endpoint")["eid_list"].to_dict()

In [15]:
from datetime import date
today = str(date.today())

In [16]:
eid_df.to_feather(f"{output_path}/eligable_eids_{today}.feather") 

In [17]:
eid_df_long = eid_df[["endpoint", "eid_list"]].explode("eid_list").reset_index(drop=True)
eid_df_long.columns = ["endpoint", "eid"]
eid_df_long["endpoint"] = eid_df_long["endpoint"].astype("category")
eid_df_long["eid"] = eid_df_long["eid"].astype("category")

In [18]:
eid_df_long.to_feather(f"{output_path}/eligable_eids_long_{today}.feather")